# Workshop 1 - Apply
    
In this notebook you will work on a homework assignment involving a Vierendeel frame.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

## Vierendeel frame

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png" WIDTH=400 ALIGN="center">
      </figure></center>

With:

- $h = 1$
- $b = 1$
- $EI_r = 10000$
- $EI_k = 1000$
- $EA  = 1\cdot 10^{10}$
- $H = 100$


In the first half of this course last quarter, you have learned that the deformation of Vierendeel frames (an example of which is shown above) can be obtained in a simplified way by assuming the global deformation can be described by a shear beam with equivalent stiffness given by:

$$
k = \frac{24}{h\left(\displaystyle\frac{h}{EI_k}+\frac{b}{EI_r}\right)}
$$

Now that you have the tools to solve the original frame problem using the Matrix Method, your task in this assignment is to investigate the validity of this equivalent shear beam model.

Note that the checks only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (Tip: refer back to what you did in the `constrain` function).

Once you have a solution, use SymPy / Maple / pen and paper to solve a shear beam problem with the equivalent stiffness given above (It is very similar to the simple extension problem above) and compare the horizontal displacement at the point of application of $H$ for the two models.

Investigate how the two models compare for different values of $EA$, ranging from very small (*e.g.* $1\cdot 10^{-5}$) to very large (*e.g.* $1\cdot10^{10}$). What explains the behavior you observe?

In [2]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [10]:
mm.Node.clear()
mm.Element.clear()

In [11]:
h = 1 
b = 1 
EI_r = 10000
EI_k = 1000 
EA = 1e10
H = 100

node1 = mm.Node(0,0)
node2 = mm.Node(b,0)
node3 = mm.Node(b,h)
node4 = mm.Node(0,h)

elem1 = mm.Element(node1, node2)
section = {}
section['EI'] = EI_r
elem1.set_section(section)
elem2 = mm.Element(node2, node3)
section = {}
section['EI'] = EI_k
elem2.set_section(section)
elem3 = mm.Element(node3, node4)
section = {}
section['EI'] = EI_r
elem3.set_section(section)
elem4 = mm.Element(node4, node1)
section = {}
section['EI'] = EI_k
elem4.set_section(section)

elems = [elem1, elem2, elem3, elem4]

In [ ]:
n_dof = 12

global_k = np.zeros((n_dof, n_dof))
global_f = np.zeros(n_dof)

for elem in elems:
    elmat = elem.stiffness()
    idofs = elem.global_dofs()
    global_k[np.ix_(idofs, idofs)] += elmat
    
    #YOUR CODE HERE

global_f[node4.dofs[0]] = H 
#for node in nodes:
    #YOUR CODE HERE

[0 1 2 3 4 5]
[3 4 5 6 7 8]
[ 6  7  8  9 10 11]
[ 9 10 11  0  1  2]


In [15]:
const = mm.Constrainer()

const.fix_dof(node1,0)
const.fix_dof(node2,0)
const.fix_dof(node1,1)
const.fix_dof(node2,2)

K_ff, f_ff = const.constrain(global_k, global_f)
u_free = np.linalg.solve(K_ff, f_ff)

u_full = const.full_disp(u_free)

print(u_full)

[ 0.00000000e+00  0.00000000e+00  4.18928682e-04  0.00000000e+00
 -3.07276291e-04  0.00000000e+00  3.26210801e-03 -3.07276291e-04
  6.04803750e-04  3.26210801e-03  1.72950773e-19  5.69893027e-04]


In [ ]:
#provided in case you want to solve the shear beam problem using SymPy
import sympy as sym
x, k, L, H = sym.symbols('x, k, L, H')
w = sym.Function('w')

ODE_shear = #YOUR CODE HERE